In [ ]:
import arcpy

class Toolbox(object):
    def __init__(self):
        """Define the toolbox (the name of the toolbox is the name of the
        .pyt file)."""
        self.label = "Solar Power Estimation Tools"
        self.alias = "toolbox"

        # List of tool classes associated with this toolbox
        self.tools = [SPET]

#Question 1
        
class SPET(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "SPET"
        self.description = "A Solar Power Estimation Tool"
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(displayName = "Select an elevation raster", name = "Raster input", datatype = "DERasterDataset", parameterType = "Required", direction = "Input")
        #param1 = arcpy.Parameter(displayName = "Select a building footprint feature class", name = "inputFC", datatype = "DEFeatureClass", parameterType = "Required", direction = "Input")
        params = [param0]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter.  This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        #from arcpy.sa import
        arcpy.CheckOutExtension("Spatial")
        arcpy.env.workspace = "C:\\Users\\Samuel\\Documents\\GEOG_592\\MiniProject\\Solar_in_Glover.gdb"
        arcpy.env.overwriteOutput = True
        
        # Add the parameter to a tool's parameter list
        #arcpy.AddToolbox("C:\\Users\\Samuel\\Documents\\GEOG_592\\MiniProject\\SolarPowerEstimation.pyt")
        #arcpy.SolarPowerEstimation.SPET(raster_input=param0)
        
        # Check out any necessary licenses.
        arcpy.CheckOutExtension("spatial")
        arcpy.CheckOutExtension("ImageAnalyst")
        arcpy.CheckOutExtension("3D")
        arcpy.CheckOutExtension("ImageExt")
        
        # Set Geoprocessing environments
        #arcpy.env.snapRaster = "C:\\Users\\Samuel\\Documents\\GEOG_592\\Lab5\\SuitabilityModeling\\Data.gdb\\Elevation"
        #arcpy.env.extent = "439952.113762345 200181.284694512 513122.113762345 253671.284694512"
        #arcpy.env.cellSize = "30"
        #arcpy.env.mask = "C:\\Users\\Samuel\\Documents\\GEOG_592\\Lab5\\SuitabilityModeling\\Data.gdb\\Elevation"
        Streams = "Streams"
        DSM = arcpy.Raster(parameters[0].ValueAsText)
                
        # Process: Area Solar Radiation (Area Solar Radiation) (sa)
        Solar_Rad_Whm2 = "Solar_Rad_Whm2"
        Area_Solar_Radiation = Solar_Rad_Whm2
        Output_direct_radiation_raster = ""
        Output_diffuse_radiation_raster = ""
        Output_direct_duration_raster = ""
        Solar_Rad_Whm2 = arcpy.sa.AreaSolarRadiation(in_surface_raster=DSM, latitude=38.92157611011734, sky_size=200, 
                                                         time_configuration="MultiDays 2023 5 10", day_interval=30, 
                                                         hour_interval=1, each_interval="NOINTERVAL", z_factor=1, 
                                                         slope_aspect_input_type="FROM_DEM", calculation_directions=16, 
                                                         zenith_divisions=8, azimuth_divisions=8, diffuse_model_type="UNIFORM_SKY", 
                                                         diffuse_proportion=0.3, transmittivity=0.5, 
                                                         out_direct_radiation_raster=Output_direct_radiation_raster, 
                                                         out_diffuse_radiation_raster=Output_diffuse_radiation_raster, 
                                                         out_direct_duration_raster=Output_direct_duration_raster)
        Solar_Rad_Whm2.save(Area_Solar_Radiation)


        # Process: Surface Parameters (2) (Surface Parameters) (sa)
        Aspect_DSM = "Aspect_DSM"
        Surface_Parameters_2_ = Aspect_DSM
        Aspect_DSM = arcpy.sa.SurfaceParameters(in_raster=DSM, parameter_type="ASPECT", local_surface_type="QUADRATIC", 
                                                    neighborhood_distance="0.5 Meters", use_adaptive_neighborhood="FIXED_NEIGHBORHOOD", 
                                                    z_unit="METER", output_slope_measurement="DEGREE", project_geodesic_azimuths="GEODESIC_AZIMUTHS", 
                                                    use_equatorial_aspect="NORTH_POLE_ASPECT")
        Aspect_DSM.save(Surface_Parameters_2_)


        # Process: Surface Parameters (Surface Parameters) (sa)
        Slope_DSM = "Slope_DSM"
        Surface_Parameters = Slope_DSM
        Slope_DSM = arcpy.sa.SurfaceParameters(in_raster=DSM, parameter_type="SLOPE", local_surface_type="QUADRATIC", 
                                                   neighborhood_distance="0.5 Meters", use_adaptive_neighborhood="FIXED_NEIGHBORHOOD", 
                                                   z_unit="METER", output_slope_measurement="DEGREE", project_geodesic_azimuths="GEODESIC_AZIMUTHS", 
                                                   use_equatorial_aspect="NORTH_POLE_ASPECT")
        Slope_DSM.save(Surface_Parameters)


        # Process: Raster Calculator (Raster Calculator) (ia)
        Solar_Rad = "Solar_Rad"
        Raster_Calculator = Solar_Rad
        Solar_Rad =  Solar_Rad_Whm2 /1000
        Solar_Rad.save(Raster_Calculator)


        # Process: Con (Con) (sa)
        Solar_Rad_slope = "Solar_Rad_slope"
        Con = Solar_Rad_slope
        Solar_Rad_slope = arcpy.sa.Con(in_conditional_raster=Slope_DSM, in_true_raster_or_constant=Solar_Rad, 
                                           in_false_raster_or_constant="", where_clause="VALUE <= 45")
        Solar_Rad_slope.save(Con)


        # Process: Con (2) (Con) (sa)
        Solar_Rad_slope_HS = "Solar_Rad_slope_HS"
        Con_2_ = Solar_Rad_slope_HS
        Solar_Rad_slope_HS = arcpy.sa.Con(in_conditional_raster=Solar_Rad_slope, in_true_raster_or_constant=Solar_Rad_slope, 
                                              in_false_raster_or_constant="", where_clause="VALUE >= 800")
        Solar_Rad_slope_HS.save(Con_2_)


        # Process: Con (3) (Con) (sa)
        Solar_Rad_Low_Slope = "Solar_Rad_Low_Slope"
        Con_3_ = Solar_Rad_Low_Slope
        Solar_Rad_Low_Slope = arcpy.sa.Con(in_conditional_raster=Slope_DSM, in_true_raster_or_constant=Solar_Rad_slope_HS, 
                                               in_false_raster_or_constant="", where_clause="VALUE <= 10")
        Solar_Rad_Low_Slope.save(Con_3_)


        # Process: Con (4) (Con) (sa)
        Solar_Rad_slope_HS_NN = "Solar_Rad_slope_HS_NN"
        Con_4_ = Solar_Rad_slope_HS_NN
        Solar_Rad_slope_HS_NN = arcpy.sa.Con(in_conditional_raster=Aspect_DSM, in_true_raster_or_constant=Solar_Rad_slope_HS, 
                                                 in_false_raster_or_constant=Solar_Rad_Low_Slope, 
                                                 where_clause="VALUE > 22.5 And VALUE < 337.5")
        Solar_Rad_slope_HS_NN.save(Con_4_)

        arcpy.AddMessage("Condition raster created successfully")
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return

In [6]:
import arcpy

# Import the toolbox
toolbox = arcpy.ImportToolbox("C:\\Users\\Samuel\\Documents\\GEOG_592\\MiniProject\\SolarPowerEstimation.pyt")

# List all the tools in the toolbox
#tools = arcpy.ListTools(toolbox)
#print(tools)

In [7]:
toolbox

<module 'toolbox'>

In [ ]:
import arcpy

class Toolbox(object):
    def __init__(self):
        """Define the toolbox (the name of the toolbox is the name of the
        .pyt file)."""
        self.label = "Solar Power Estimation Tools"
        self.alias = "toolbox"

        # List of tool classes associated with this toolbox
        self.tools = [SPET]

#Question 1
        
class SPET(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "SPET"
        self.description = "A Solar Power Estimation Tool"
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(displayName = "Select an elevation raster", name = "Raster input", datatype = "DERasterDataset", parameterType = "Required", direction = "Input")
        #param1 = arcpy.Parameter(displayName = "Select a building footprint feature class", name = "inputFC", datatype = "DEFeatureClass", parameterType = "Required", direction = "Input")
        params = [param0]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter.  This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        #from arcpy.sa import
        arcpy.CheckOutExtension("Spatial")
        arcpy.env.workspace = "C:\\Users\\Samuel\\Documents\\GEOG_592\\MiniProject\\Solar_in_Glover.gdb"
        arcpy.env.overwriteOutput = True
        
        # Add the parameter to a tool's parameter list
        #arcpy.AddToolbox("C:\\Users\\Samuel\\Documents\\GEOG_592\\MiniProject\\SolarPowerEstimation.pyt")
        #arcpy.SolarPowerEstimation.SPET(raster_input=param0)
        
        # Check out any necessary licenses.
        arcpy.CheckOutExtension("spatial")
        arcpy.CheckOutExtension("ImageAnalyst")
        arcpy.CheckOutExtension("3D")
        arcpy.CheckOutExtension("ImageExt")
        
        # Set Geoprocessing environments
        #arcpy.env.snapRaster = "C:\\Users\\Samuel\\Documents\\GEOG_592\\Lab5\\SuitabilityModeling\\Data.gdb\\Elevation"
        #arcpy.env.extent = "439952.113762345 200181.284694512 513122.113762345 253671.284694512"
        #arcpy.env.cellSize = "30"
        #arcpy.env.mask = "C:\\Users\\Samuel\\Documents\\GEOG_592\\Lab5\\SuitabilityModeling\\Data.gdb\\Elevation"
        Streams = "Streams"
        DSM = arcpy.Raster(parameters[0].ValueAsText)
                
        # Process: Area Solar Radiation (Area Solar Radiation) (sa)
        Solar_Rad_Whm2 = "Solar_Rad_Whm2"
        Area_Solar_Radiation = Solar_Rad_Whm2
        Output_direct_radiation_raster = ""
        Output_diffuse_radiation_raster = ""
        Output_direct_duration_raster = ""
        Solar_Rad_Whm2 = arcpy.sa.AreaSolarRadiation(in_surface_raster=DSM, latitude=38.92157611011734, sky_size=200, 
                                                         time_configuration="MultiDays 2023 5 10", day_interval=30, 
                                                         hour_interval=1, each_interval="NOINTERVAL", z_factor=1, 
                                                         slope_aspect_input_type="FROM_DEM", calculation_directions=16, 
                                                         zenith_divisions=8, azimuth_divisions=8, diffuse_model_type="UNIFORM_SKY", 
                                                         diffuse_proportion=0.3, transmittivity=0.5, 
                                                         out_direct_radiation_raster=Output_direct_radiation_raster, 
                                                         out_diffuse_radiation_raster=Output_diffuse_radiation_raster, 
                                                         out_direct_duration_raster=Output_direct_duration_raster)
        Solar_Rad_Whm2.save(Area_Solar_Radiation)


        # Process: Surface Parameters (2) (Surface Parameters) (sa)
        Aspect_DSM = "Aspect_DSM"
        Surface_Parameters_2_ = Aspect_DSM
        Aspect_DSM = arcpy.sa.SurfaceParameters(in_raster=DSM, parameter_type="ASPECT", local_surface_type="QUADRATIC", 
                                                    neighborhood_distance="0.5 Meters", use_adaptive_neighborhood="FIXED_NEIGHBORHOOD", 
                                                    z_unit="METER", output_slope_measurement="DEGREE", project_geodesic_azimuths="GEODESIC_AZIMUTHS", 
                                                    use_equatorial_aspect="NORTH_POLE_ASPECT")
        Aspect_DSM.save(Surface_Parameters_2_)


        # Process: Surface Parameters (Surface Parameters) (sa)
        Slope_DSM = "Slope_DSM"
        Surface_Parameters = Slope_DSM
        Slope_DSM = arcpy.sa.SurfaceParameters(in_raster=DSM, parameter_type="SLOPE", local_surface_type="QUADRATIC", 
                                                   neighborhood_distance="0.5 Meters", use_adaptive_neighborhood="FIXED_NEIGHBORHOOD", 
                                                   z_unit="METER", output_slope_measurement="DEGREE", project_geodesic_azimuths="GEODESIC_AZIMUTHS", 
                                                   use_equatorial_aspect="NORTH_POLE_ASPECT")
        Slope_DSM.save(Surface_Parameters)


        # Process: Raster Calculator (Raster Calculator) (ia)
        Solar_Rad = "Solar_Rad"
        Raster_Calculator = Solar_Rad
        Solar_Rad =  Solar_Rad_Whm2 /1000
        Solar_Rad.save(Raster_Calculator)


        # Process: Con (Con) (sa)
        Solar_Rad_slope = "Solar_Rad_slope"
        Con = Solar_Rad_slope
        Solar_Rad_slope = arcpy.sa.Con(in_conditional_raster=Slope_DSM, in_true_raster_or_constant=Solar_Rad, 
                                           in_false_raster_or_constant="", where_clause="VALUE <= 45")
        Solar_Rad_slope.save(Con)


        # Process: Con (2) (Con) (sa)
        Solar_Rad_slope_HS = "Solar_Rad_slope_HS"
        Con_2_ = Solar_Rad_slope_HS
        Solar_Rad_slope_HS = arcpy.sa.Con(in_conditional_raster=Solar_Rad_slope, in_true_raster_or_constant=Solar_Rad_slope, 
                                              in_false_raster_or_constant="", where_clause="VALUE >= 800")
        Solar_Rad_slope_HS.save(Con_2_)


        # Process: Con (3) (Con) (sa)
        Solar_Rad_Low_Slope = "Solar_Rad_Low_Slope"
        Con_3_ = Solar_Rad_Low_Slope
        Solar_Rad_Low_Slope = arcpy.sa.Con(in_conditional_raster=Slope_DSM, in_true_raster_or_constant=Solar_Rad_slope_HS, 
                                               in_false_raster_or_constant="", where_clause="VALUE <= 10")
        Solar_Rad_Low_Slope.save(Con_3_)


        # Process: Con (4) (Con) (sa)
        Solar_Rad_slope_HS_NN = "Solar_Rad_slope_HS_NN"
        Con_4_ = Solar_Rad_slope_HS_NN
        Solar_Rad_slope_HS_NN = arcpy.sa.Con(in_conditional_raster=Aspect_DSM, in_true_raster_or_constant=Solar_Rad_slope_HS, 
                                                 in_false_raster_or_constant=Solar_Rad_Low_Slope, 
                                                 where_clause="VALUE > 22.5 And VALUE < 337.5")
        Solar_Rad_slope_HS_NN.save(Con_4_)

        arcpy.AddMessage("Condition raster created successfully")
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return

In [ ]:
import arcpy

class Toolbox(object):
    def __init__(self):
        """Define the toolbox (the name of the toolbox is the name of the
        .pyt file)."""
        self.label = "Solar Power Estimation Tools"
        self.alias = "toolbox"

        # List of tool classes associated with this toolbox
        self.tools = [SPET]

#Question 1
        
class SPET(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "SPET"
        self.description = "A Solar Power Estimation Tool"
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(displayName = "Select an elevation raster", name = "Raster input", datatype = "DERasterDataset", parameterType = "Required", direction = "Input")
        param1 = arcpy.Parameter(displayName = "Specify hour interval", name = "time_intrv", datatype = "GPLong", parameterType = "Required", direction = "Input")
        param2 = arcpy.Parameter(displayName = "Calculation directions", name = "cal dir", datatype = "GPLong", parameterType = "Required", direction = "Input")
        param3 = arcpy.Parameter(displayName = "Local surface type", name = "surface type", datatype = "GPString", parameterType = "Required", direction = "Input")
        lst_list = ['QUADRATIC', 'BIQUADRATIC']
        param3.filter.type = "ValueList"
        param3.filter.list = lst_list
        param4 = arcpy.Parameter(displayName = "Neighborhood distance", name = "n_dist", datatype = "GPLinearUnit", parameterType = "Required", direction = "Input")
        param5 = arcpy.Parameter(displayName = "output slope measurement", name = "out_slope", datatype = "GPString", parameterType = "Required", direction = "Input")
        osm_list = ['DEGREE', 'PERCENT_RISE']
        param5.filter.type = "ValueList"
        param5.filter.list = osm_list
        param6 = arcpy.Parameter(displayName = "Threshold for Steep slope", name = "StpSlopeThrhld", datatype = "GPLong", parameterType = "Required", direction = "Input")
        param6.filter.type = "Range"
        param6.filter.list = [0, 90]
        param7 = arcpy.Parameter(displayName = "Minimum solar radiation", name = "MinSolRad", datatype = "GPLong", parameterType = "Required", direction = "Input")
        param8 = arcpy.Parameter(displayName = "Threshold for flat slope", name = "FltSlopeThrhld", datatype = "GPLong", parameterType = "Required", direction = "Input")
        param8.filter.type = "Range"
        param8.filter.list = [0, 90]
        #param0 = arcpy.Parameter(displayName = "Range for North-facing slopes", name = "NFSlopeThrhld", datatype = "GPLong", parameterType = "Required", direction = "Input")
        param9 = arcpy.Parameter(displayName = "Select a building footprint feature class", name = "inputFC", datatype = "DEFeatureClass", parameterType = "Required", direction = "Input")
        param10 = arcpy.Parameter(displayName = "Select the Zone field", name = "Zonefld", datatype = "Field", parameterType = "Required", direction = "Input")
        param10.parameterDependencies = [param6.name]
        params = [param0, param1, param2, param3, param4, param5, param6, param7, param8, param9, param]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter.  This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        #from arcpy.sa import
        arcpy.CheckOutExtension("Spatial")
        arcpy.env.workspace = "C:\\Users\\Samuel\\Documents\\GEOG_592\\MiniProject\\Solar_in_Glover.gdb"
        arcpy.env.overwriteOutput = True
        
        # Add the parameter to a tool's parameter list
        #arcpy.AddToolbox("C:\\Users\\Samuel\\Documents\\GEOG_592\\MiniProject\\SolarPowerEstimation.pyt")
        #arcpy.SolarPowerEstimation.SPET(raster_input=param0)
        
        # Check out any necessary licenses.
        arcpy.CheckOutExtension("spatial")
        arcpy.CheckOutExtension("ImageAnalyst")
        arcpy.CheckOutExtension("3D")
        arcpy.CheckOutExtension("ImageExt")
        
        # Set Geoprocessing environments
        #arcpy.env.snapRaster = "C:\\Users\\Samuel\\Documents\\GEOG_592\\Lab5\\SuitabilityModeling\\Data.gdb\\Elevation"
        #arcpy.env.extent = "439952.113762345 200181.284694512 513122.113762345 253671.284694512"
        #arcpy.env.cellSize = "30"
        #arcpy.env.mask = "C:\\Users\\Samuel\\Documents\\GEOG_592\\Lab5\\SuitabilityModeling\\Data.gdb\\Elevation"
        
        DSM = arcpy.Raster(parameters[0].ValueAsText)
                
        # Process: Area Solar Radiation (Area Solar Radiation) (sa)
        Solar_Rad_Whm2 = "Solar_Rad_Whm2"
        Area_Solar_Radiation = Solar_Rad_Whm2
        Output_direct_radiation_raster = ""
        Output_diffuse_radiation_raster = ""
        Output_direct_duration_raster = ""
        Solar_Rad_Whm2 = arcpy.sa.AreaSolarRadiation(in_surface_raster=DSM, latitude=38.92157611011734, sky_size=200, 
                                                         time_configuration="WholeYear 2023", day_interval=14, 
                                                         hour_interval=12, each_interval="NOINTERVAL", z_factor=1, 
                                                         slope_aspect_input_type="FROM_DEM", calculation_directions=parameters[2].ValueAsText, 
                                                         zenith_divisions=8, azimuth_divisions=8, diffuse_model_type="UNIFORM_SKY", 
                                                         diffuse_proportion=0.3, transmittivity=0.5, 
                                                         out_direct_radiation_raster=Output_direct_radiation_raster, 
                                                         out_diffuse_radiation_raster=Output_diffuse_radiation_raster, 
                                                         out_direct_duration_raster=Output_direct_duration_raster)
        Solar_Rad_Whm2.save(Area_Solar_Radiation)


        # Process: Surface Parameters (2) (Surface Parameters) (sa)
        Aspect_DSM = "Aspect_DSM"
        Surface_Parameters_2_ = Aspect_DSM
        Aspect_DSM = arcpy.sa.SurfaceParameters(in_raster=DSM, parameter_type="ASPECT", local_surface_type=parameters[3].ValueAsText, 
                                                    neighborhood_distance=parameters[4].ValueAsText, use_adaptive_neighborhood="FIXED_NEIGHBORHOOD", 
                                                    z_unit="METER", output_slope_measurement=parameters[5].ValueAsText, project_geodesic_azimuths="GEODESIC_AZIMUTHS", 
                                                    use_equatorial_aspect="NORTH_POLE_ASPECT")
        Aspect_DSM.save(Surface_Parameters_2_)


        # Process: Surface Parameters (Surface Parameters) (sa)
        Slope_DSM = "Slope_DSM"
        Surface_Parameters = Slope_DSM
        Slope_DSM = arcpy.sa.SurfaceParameters(in_raster=DSM, parameter_type="SLOPE", local_surface_type=parameters[3].ValueAsText, 
                                                   neighborhood_distance=parameters[4].ValueAsText, use_adaptive_neighborhood="FIXED_NEIGHBORHOOD", 
                                                   z_unit="METER", output_slope_measurement=parameters[5].ValueAsText, project_geodesic_azimuths="GEODESIC_AZIMUTHS", 
                                                   use_equatorial_aspect="NORTH_POLE_ASPECT")
        Slope_DSM.save(Surface_Parameters)


        # Process: Raster Calculator (Raster Calculator) (ia)
        Solar_Rad = "Solar_Rad"
        Raster_Calculator = Solar_Rad
        Solar_Rad =  Solar_Rad_Whm2 /1000
        Solar_Rad.save(Raster_Calculator)


        # Process: Con (Con) (sa)
        Solar_Rad_slope = "Solar_Rad_slope"
        Con = Solar_Rad_slope
        Solar_Rad_slope = arcpy.sa.Con(in_conditional_raster=Slope_DSM, in_true_raster_or_constant=Solar_Rad, 
                                           in_false_raster_or_constant="", where_clause="VALUE <= 45")
        Solar_Rad_slope.save(Con)


        # Process: Con (2) (Con) (sa)
        Solar_Rad_slope_HS = "Solar_Rad_slope_HS"
        Con_2_ = Solar_Rad_slope_HS
        Solar_Rad_slope_HS = arcpy.sa.Con(in_conditional_raster=Solar_Rad_slope, in_true_raster_or_constant=Solar_Rad_slope, 
                                              in_false_raster_or_constant="", where_clause="VALUE >= 800")
        Solar_Rad_slope_HS.save(Con_2_)


        # Process: Con (3) (Con) (sa)
        Solar_Rad_Low_Slope = "Solar_Rad_Low_Slope"
        Con_3_ = Solar_Rad_Low_Slope
        Solar_Rad_Low_Slope = arcpy.sa.Con(in_conditional_raster=Slope_DSM, in_true_raster_or_constant=Solar_Rad_slope_HS, 
                                               in_false_raster_or_constant="", where_clause="VALUE <= 10")
        Solar_Rad_Low_Slope.save(Con_3_)


        # Process: Con (4) (Con) (sa)
        Solar_Rad_slope_HS_NN = "Solar_Rad_slope_HS_NN"
        Con_4_ = Solar_Rad_slope_HS_NN
        Solar_Rad_slope_HS_NN = arcpy.sa.Con(in_conditional_raster=Aspect_DSM, in_true_raster_or_constant=Solar_Rad_slope_HS, 
                                                 in_false_raster_or_constant=Solar_Rad_Low_Slope, 
                                                 where_clause="VALUE > 22.5 And VALUE < 337.5")
        Solar_Rad_slope_HS_NN.save(Con_4_)

        # Process: Zonal Statistics as Table (Zonal Statistics as Table) (sa)
        ZonalSt_Buildin1 = "C:\\Users\\Samuel\\Documents\\GEOG_592\\MiniProject\\GISC_Project.gdb\\ZonalSt_Buildin1"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=parameters[9].ValueAsText, zone_field=parameters[10].ValueAsText, in_value_raster=Solar_Rad_slope_HS_NN, out_table=ZonalSt_Buildin1, ignore_nodata="DATA", statistics_type="MEAN", process_as_multidimensional="CURRENT_SLICE", percentile_values=90, percentile_interpolation_type="AUTO_DETECT", circular_calculation="ARITHMETIC", circular_wrap_value=360)
        arcpy.TableToTable_conversion(in_rows=ZonalSt_Buildin1, out_path="C:\\Users\\Samuel\\Documents\\GEOG_592\\MiniProject\\GISC_Project.gdb", out_name="Zonal_Statistics_as_Table")

        arcpy.AddMessage("Condition raster created successfully")
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return

#### Last Code

In [ ]:
import arcpy

class Toolbox(object):
    def __init__(self):
        """Define the toolbox (the name of the toolbox is the name of the
        .pyt file)."""
        self.label = "Solar Power Estimation Tools"
        self.alias = "toolbox"

        # List of tool classes associated with this toolbox
        self.tools = [SPET]

#Question 1
        
class SPET(object):
    def __init__(self):
        """Define the tool (tool name is the name of the class)."""
        self.label = "SPET"
        self.description = "A Solar Power Estimation Tool"
        self.canRunInBackground = False

    def getParameterInfo(self):
        """Define parameter definitions"""
        param0 = arcpy.Parameter(displayName = "Select an elevation raster", name = "Raster input", datatype = "DERasterDataset", parameterType = "Required", direction = "Input")
        param1 = arcpy.Parameter(displayName = "Specify hour interval", name = "time_intrv", datatype = "GPLong", parameterType = "Required", direction = "Input")
        param2 = arcpy.Parameter(displayName = "Calculation directions", name = "cal dir", datatype = "GPLong", parameterType = "Required", direction = "Input")
        param3 = arcpy.Parameter(displayName = "Local surface type", name = "surface type", datatype = "GPString", parameterType = "Required", direction = "Input")
        lst_list = ['QUADRATIC', 'BIQUADRATIC']
        param3.filter.type = "ValueList"
        param3.filter.list = lst_list
        param4 = arcpy.Parameter(displayName = "Neighborhood distance", name = "n_dist", datatype = "GPLinearUnit", parameterType = "Required", direction = "Input")
        param5 = arcpy.Parameter(displayName = "output slope measurement", name = "out_slope", datatype = "GPString", parameterType = "Required", direction = "Input")
        osm_list = ['DEGREE', 'PERCENT_RISE']
        param5.filter.type = "ValueList"
        param5.filter.list = osm_list
        params = [param0, param1, param2, param3, param4, param5]
        return params

    def isLicensed(self):
        """Set whether tool is licensed to execute."""
        return True

    def updateParameters(self, parameters):
        """Modify the values and properties of parameters before internal
        validation is performed.  This method is called whenever a parameter
        has been changed."""
        return

    def updateMessages(self, parameters):
        """Modify the messages created by internal validation for each tool
        parameter.  This method is called after internal validation."""
        return

    def execute(self, parameters, messages):
        """The source code of the tool."""
        #from arcpy.sa import
        arcpy.CheckOutExtension("Spatial")
        arcpy.env.workspace = "C:\\Users\\Samuel\\Documents\\GEOG_592\\MiniProject\\Solar_in_Glover.gdb"
        arcpy.env.overwriteOutput = True
        
        # Add the parameter to a tool's parameter list
        #arcpy.AddToolbox("C:\\Users\\Samuel\\Documents\\GEOG_592\\MiniProject\\SolarPowerEstimation.pyt")
        #arcpy.SolarPowerEstimation.SPET(raster_input=param0)
        
        # Check out any necessary licenses.
        arcpy.CheckOutExtension("spatial")
        arcpy.CheckOutExtension("ImageAnalyst")
        arcpy.CheckOutExtension("3D")
        arcpy.CheckOutExtension("ImageExt")
        
        # Set Geoprocessing environments
        #arcpy.env.snapRaster = "C:\\Users\\Samuel\\Documents\\GEOG_592\\Lab5\\SuitabilityModeling\\Data.gdb\\Elevation"
        #arcpy.env.extent = "439952.113762345 200181.284694512 513122.113762345 253671.284694512"
        #arcpy.env.cellSize = "30"
        #arcpy.env.mask = "C:\\Users\\Samuel\\Documents\\GEOG_592\\Lab5\\SuitabilityModeling\\Data.gdb\\Elevation"
        
        DSM = arcpy.Raster(parameters[0].ValueAsText)
                
        # Process: Area Solar Radiation (Area Solar Radiation) (sa)
        Solar_Rad_Whm2 = "Solar_Rad_Whm2"
        Area_Solar_Radiation = Solar_Rad_Whm2
        Output_direct_radiation_raster = ""
        Output_diffuse_radiation_raster = ""
        Output_direct_duration_raster = ""
        Solar_Rad_Whm2 = arcpy.sa.AreaSolarRadiation(in_surface_raster=DSM, latitude=38.92157611011734, sky_size=200, 
                                                         time_configuration="WholeYear 2023", day_interval=14, 
                                                         hour_interval=12, each_interval="NOINTERVAL", z_factor=1, 
                                                         slope_aspect_input_type="FROM_DEM", calculation_directions=parameters[2].ValueAsText, 
                                                         zenith_divisions=8, azimuth_divisions=8, diffuse_model_type="UNIFORM_SKY", 
                                                         diffuse_proportion=0.3, transmittivity=0.5, 
                                                         out_direct_radiation_raster=Output_direct_radiation_raster, 
                                                         out_diffuse_radiation_raster=Output_diffuse_radiation_raster, 
                                                         out_direct_duration_raster=Output_direct_duration_raster)
        Solar_Rad_Whm2.save(Area_Solar_Radiation)


        # Process: Surface Parameters (2) (Surface Parameters) (sa)
        Aspect_DSM = "Aspect_DSM"
        Surface_Parameters_2_ = Aspect_DSM
        Aspect_DSM = arcpy.sa.SurfaceParameters(in_raster=DSM, parameter_type="ASPECT", local_surface_type=parameters[3].ValueAsText, 
                                                    neighborhood_distance=parameters[4].ValueAsText, use_adaptive_neighborhood="FIXED_NEIGHBORHOOD", 
                                                    z_unit="METER", output_slope_measurement=parameters[5].ValueAsText, project_geodesic_azimuths="GEODESIC_AZIMUTHS", 
                                                    use_equatorial_aspect="NORTH_POLE_ASPECT")
        Aspect_DSM.save(Surface_Parameters_2_)


        # Process: Surface Parameters (Surface Parameters) (sa)
        Slope_DSM = "Slope_DSM"
        Surface_Parameters = Slope_DSM
        Slope_DSM = arcpy.sa.SurfaceParameters(in_raster=DSM, parameter_type="SLOPE", local_surface_type=parameters[3].ValueAsText, 
                                                   neighborhood_distance=parameters[4].ValueAsText, use_adaptive_neighborhood="FIXED_NEIGHBORHOOD", 
                                                   z_unit="METER", output_slope_measurement=parameters[5].ValueAsText, project_geodesic_azimuths="GEODESIC_AZIMUTHS", 
                                                   use_equatorial_aspect="NORTH_POLE_ASPECT")
        Slope_DSM.save(Surface_Parameters)


        # Process: Raster Calculator (Raster Calculator) (ia)
        Solar_Rad = "Solar_Rad"
        Raster_Calculator = Solar_Rad
        Solar_Rad =  Solar_Rad_Whm2 /1000
        Solar_Rad.save(Raster_Calculator)


        # Process: Con (Con) (sa)
        Solar_Rad_slope = "Solar_Rad_slope"
        Con = Solar_Rad_slope
        Solar_Rad_slope = arcpy.sa.Con(in_conditional_raster=Slope_DSM, in_true_raster_or_constant=Solar_Rad, 
                                           in_false_raster_or_constant="", where_clause="VALUE <= 45")
        Solar_Rad_slope.save(Con)


        # Process: Con (2) (Con) (sa)
        Solar_Rad_slope_HS = "Solar_Rad_slope_HS"
        Con_2_ = Solar_Rad_slope_HS
        Solar_Rad_slope_HS = arcpy.sa.Con(in_conditional_raster=Solar_Rad_slope, in_true_raster_or_constant=Solar_Rad_slope, 
                                              in_false_raster_or_constant="", where_clause="VALUE >= 800")
        Solar_Rad_slope_HS.save(Con_2_)


        # Process: Con (3) (Con) (sa)
        Solar_Rad_Low_Slope = "Solar_Rad_Low_Slope"
        Con_3_ = Solar_Rad_Low_Slope
        Solar_Rad_Low_Slope = arcpy.sa.Con(in_conditional_raster=Slope_DSM, in_true_raster_or_constant=Solar_Rad_slope_HS, 
                                               in_false_raster_or_constant="", where_clause="VALUE <= 10")
        Solar_Rad_Low_Slope.save(Con_3_)


        # Process: Con (4) (Con) (sa)
        Solar_Rad_slope_HS_NN = "Solar_Rad_slope_HS_NN"
        Con_4_ = Solar_Rad_slope_HS_NN
        Solar_Rad_slope_HS_NN = arcpy.sa.Con(in_conditional_raster=Aspect_DSM, in_true_raster_or_constant=Solar_Rad_slope_HS, 
                                                 in_false_raster_or_constant=Solar_Rad_Low_Slope, 
                                                 where_clause="VALUE > 22.5 And VALUE < 337.5")
        Solar_Rad_slope_HS_NN.save(Con_4_)

        arcpy.AddMessage("Condition raster created successfully")
        return

    def postExecute(self, parameters):
        """This method takes place after outputs are processed and
        added to the display."""
        return

In [ ]:
 with arcpy.da.SearchCursor(parameters[0].ValueAsText, ["NAME", "STATE_NAME"]) as cursor:
        for row in cursor:
            if param4 = row[1]:
                